In [11]:
from pathlib import Path

class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        children = sorted(list(path
                               for path in root.iterdir()
                               if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))

In [19]:
# With a criteria (skip hidden files)
def is_not_hidden(path):
    return not path.name.startswith(".")

paths = DisplayablePath.make_tree(Path('./experiment1/'), criteria=is_not_hidden)
for path in paths:
    print(path.displayable())

experiment1/
├── .DS_Store
├── input_parameters.prp
├── inputs/
│   ├── inputs1.in
│   ├── inputs10.in
│   ├── inputs11.in
│   ├── inputs12.in
│   ├── inputs13.in
│   ├── inputs14.in
│   ├── inputs15.in
│   ├── inputs2.in
│   ├── inputs3.in
│   ├── inputs4.in
│   ├── inputs5.in
│   ├── inputs6.in
│   ├── inputs7.in
│   ├── inputs8.in
│   └── inputs9.in
├── inputs.in
├── output.out
├── scheduler/
│   ├── FCFS/
│   │   ├── output/
│   │   │   ├── output1.out
│   │   │   ├── output10.out
│   │   │   ├── output11.out
│   │   │   ├── output12.out
│   │   │   ├── output13.out
│   │   │   ├── output14.out
│   │   │   ├── output15.out
│   │   │   ├── output2.out
│   │   │   ├── output3.out
│   │   │   ├── output4.out
│   │   │   ├── output5.out
│   │   │   ├── output6.out
│   │   │   ├── output7.out
│   │   │   ├── output8.out
│   │   │   └── output9.out
│   │   └── simulator_parameters.prp
│   ├── ISJF/
│   │   ├── output/
│   │   │   ├── output1.out
│   │   │   ├── output10.out
│   │   │   ├